# Modèle 1 : Finalisation dernière minute

- On ajoute des poids pour bénéficier "Définition" \*1.3 et réduire "Titre" \*0.7

## Imports

In [ ]:
!pip install pandas spacy bs4 sentence_transformers numpy
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 3.5 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [3]:
import pandas as pd
import re
import unicodedata
import spacy
from bs4 import BeautifulSoup
from spacy.lang.fr.stop_words import STOP_WORDS
import pickle
from sentence_transformers import SentenceTransformer, util
from sentence_transformers.quantization import quantize_embeddings
import numpy as np

/home/patrice/Documents/Etudes/CY-Tech/Ing2_Sem2_2023-2024/UE7_Projet_DataBattle_IA-Pau/ia-pau-data-battle-mars-2024/backend/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Anciennes fonctions

In [5]:
def calculate_average_embedding(text, quantize=False, precision="binary"):
    # Diviser le texte en phrases
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]
    
    # Calculer l'embedding de chaque phrase
    if quantize :
        sentence_embeddings = model.encode(sentences, precision=precision)
    else :
        sentence_embeddings = model.encode(sentences)
    
    # Prendre la moyenne des embeddings des phrases
    if len(sentence_embeddings) > 0:
        average_embedding = np.mean(sentence_embeddings, axis=0)
    else:
        average_embedding = np.zeros(model.get_sentence_embedding_dimension())
    
    return average_embedding

def pre_processing_sections(sections):
    clean_sections = []

    for texte in sections :
        # Nettoyer HTML Tags
        texte = BeautifulSoup(texte, 'html.parser').get_text()

        # Remplacer "&nbsp;." par rien
        texte = re.sub(r'&nbsp;\.', '', texte)

        # Accents
        texte = unicodedata.normalize('NFD', texte).encode('ascii', 'ignore').decode("utf-8")

        # Retirer les numéros
        texte = re.sub(r'\b\d+\b', '', texte)

        # Tokenization, Lemmatization, Removing Stopwords, Lowercase
        doc = nlp(texte)
        phrases_propres = []
        for phrase in doc.sents:
            tokens = [token.lemma_.lower() for token in phrase if not token.is_stop and not token.is_punct and not token.is_space]
            phrase_propre = ' '.join(tokens)
            if phrase_propre:
                phrases_propres.append(phrase_propre + "")

        # Joining the cleaned sentences back into a single string
        cleaned_text = ' '.join(phrases_propres)

        clean_sections.append(cleaned_text)

    return clean_sections

# NOUVELLE FONCTION
def calculate_section_embedding(sections):
    embeddings = []
    for section in sections :
        section_average_embedding = calculate_average_embedding(section)
        embeddings.append(section_average_embedding)
    return embeddings

def genere_embedding(data, output_file, quantize=False, precision="binary"):

    # Appliquer la fonction pour calculer l'embedding moyen à chaque texte
    if quantize:
        embeddings = data['clean_text'].apply(calculate_section_embedding, quantize=True)
    else :
        embeddings = data['clean_text'].apply(calculate_section_embedding)

    
    # Créer un nouveau DataFrame avec id_solution et les embeddings
    new_data = {
        'id_solution': data['id_solution'],
        'text_embedding': embeddings # Désormais mes embeddings sont des listes de solutions contenant chacune les vecteurs de toutes les phrases de la solution.
    }
    
    # Créer un DataFrame à partir des nouvelles données
    df_embeddings = pd.DataFrame(new_data)
    
    # Storer les embeddings dans un fichier
    with open(output_file, "wb") as fOut:
        pickle.dump(df_embeddings, fOut, protocol=pickle.HIGHEST_PROTOCOL)
    
    print("Les embeddings ont été storer avec succès.")

def find_solution(text_to_compare, embeddings_file, quantize=False, precision="binary"):
    # Calculer l'embedding moyen du texte à comparer
    embedding_to_compare = calculate_average_embedding(text_to_compare, quantize, precision)
    
    # Charger les embeddings à partir du fichier
    with open(embeddings_file, "rb") as fIn:
        df_embeddings = pickle.load(fIn)


    list_similarities = []
    # Pour chaque solution
    for solution in df_embeddings['text_embedding'].values:
        embeddings_array = np.stack(solution)

        # Calculer la similarité cosinus entre l'embedding à comparer et les embeddings dans df_embeddings
        similarities = util.pytorch_cos_sim(embedding_to_compare.reshape(1, -1).astype(np.float64), embeddings_array.astype(np.float64))

        max_value, _ = similarities.max(dim=1)
        list_similarities.append(max_value)

    # Ajouter les similarités au DataFrame df_embeddings
    df_embeddings['similarity'] = list_similarities
    
    # Trier par similarité décroissante
    df_sorted = df_embeddings.sort_values(by='similarity', ascending=False)
    
    # Récupérer les id_solution et les similarités
    solution_info = df_sorted[['id_solution', 'similarity']].head(10)
    
    # Convertir en liste de tuples (id_solution, similarity)
    solution_list = list(zip(solution_info['id_solution'], solution_info['similarity']))
    
    return solution_list

## Nouvelles fonctions

### On traite les données

In [102]:
def load_and_merge_data(csv_file='../data/solutions2.csv'):
    dictionnaire_solution = {1: "Titre", 2: "Description", 5: "Application", 6: "Bilan énergétique", 21: "Titre technologie", 22: "Description technologie"}

    # Charger le fichier CSV en spécifiant le séparateur '|'
    df = pd.read_csv(csv_file, sep='|', header=None)

    # Initialiser une liste pour stocker les données de chaque solution
    solutions_data = []

    # Parcourir chaque ligne du DataFrame
    for index, row in df.iterrows():
        id_sol = row[0]
        section = row[1]
        texte = row[2]

        # Vérifier si la section correspond à une clé dans le dictionnaire de solutions
        if section in dictionnaire_solution:
            # Récupérer le nom de la section
            section_name = dictionnaire_solution[section]

            # Chercher si la solution existe déjà dans la liste
            solution_exists = False
            for solution in solutions_data:
                if solution[0] == id_sol:
                    solution_exists = True
                    solution[1][section_name] = texte
                    break

            # Si la solution n'existe pas encore, la créer
            if not solution_exists:
                new_solution = [id_sol, {section_name: texte}]
                solutions_data.append(new_solution)

    return solutions_data

In [103]:
# Appel de la fonction pour obtenir les données
df_solutions = load_and_merge_data()
print(df_solutions[0])

[2, {'Titre technologie': 'Centrale froid', 'Titre': 'Installation frigorifique négative de type cascade utilisant du CO2', 'Description': 'Mise en place dune installation frigorifique négative de type cascade utilisant du CO2 comme fluide frigorigène.', 'Application': 'Pour être éligible à CEE, la mise en place doit être effectuée par un professionnel et appliquée dans des locaux de commerce de distribution alimentaire de surface de vente inférieure à 5000 m². Comparé aux autres fluides frigorigènes, le CO2 est un fluide'}]


In [104]:
# Charger le modèle de langue SpaCy
nlp = spacy.load("fr_core_news_sm")

def clean_df_solutions(df_solutions):
    cleaned_data = []
    for item in df_solutions:
        index = item[0]
        solution = item[1]
        cleaned_solution = {}

        for key, value in solution.items():
            if (key == 'Titre' or key == 'Titre technologie') :
                # Pour les titres, ne pas enlever les chiffres
                cleaned_solution[key] = clean_text(str(value), remove_numbers=False)
            else:
                # Pour les autres champs, enlever les chiffres
                cleaned_solution[key] = clean_text(str(value), remove_numbers=True)

        cleaned_data.append([index, cleaned_solution])

    return cleaned_data

def clean_text(text, remove_numbers=True):
    # Nettoyer HTML Tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Remplacer "&nbsp;." par rien
    text = re.sub(r'&nbsp;\.', '', text)

    # Supprimer les "l'"
    text = re.sub(r"\bl'", '', text)

    # Accents
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")

    if remove_numbers:
        # Retirer les numéros
        text = re.sub(r'\b\d+\b', '', text)

    # Supprimer les caractères seuls
    text = re.sub(r'\b\w\b', '', text)

    # Tokenization, Lemmatization, Removing Stopwords, Lowercase
    doc = nlp(text)
    cleaned_sentences = []
    for sentence in doc.sents:
        tokens = [token.lemma_.lower() for token in sentence if not token.is_stop and not token.is_punct and not token.is_space]
        clean_sentence = ' '.join(tokens)
        if clean_sentence:
            cleaned_sentences.append(clean_sentence)  # Ajouter un point à la fin de la phrase propre

    # Joining the cleaned sentences back into a single string
    cleaned_text = ' '.join(cleaned_sentences)

    return cleaned_text

In [120]:
df_solutions_clean = clean_df_solutions(df_solutions)

/tmp/ipykernel_131978/1373267169.py:25: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


In [106]:
print(df_solutions_clean)

[[2, {'Titre technologie': 'central froid', 'Titre': 'installation frigorifique negativ type cascade utiliser co2', 'Description': 'mise place dun installation frigorifique negativ type cascade utiliser co2 fluide frigorigene', 'Application': 'eligibl cee mise place effectuee professionnel appliquee local commerce distribution alimentaire surface vent inferieure comparer fluide frigorigene co2 fluide'}], [3, {'Titre technologie': 'central froid', 'Titre': 'centrale negativ mode booster', 'Description': 'linstallation centrale frigorifique bas moyenne temperatur mode booster consister injecter refoulement compresseur froid negatif laspiration compresseur cycle froid positif maniere rendement cycle frigorifique bas temperature negatif grandement ameliore condensation bas temperature egale temperature devaporation central positif solution diminuer quantite tuyauterie utilisee', 'Application': 'solution applicable uniquement projet neuf projet refonte total systeme production froid eligibl

Pour vérifier qu'on garde les numéros

In [108]:
def get_solution_by_id(df_solutions_clean, solution_id):
    for item in df_solutions_clean:
        if item[0] == solution_id:
            return item[1]
    return None

# Utilisation de la fonction pour obtenir le dictionnaire de l'ID 22
solution_id_choisi = 1692
solution_choisie = get_solution_by_id(df_solutions_clean, solution_id_choisi)

# Afficher le dictionnaire de la solution choisie
print(solution_choisie)

{'Titre technologie': 'recyclage', 'Description technologie': 'recyclage dechet suite procede permettre valorisation matier dechet industriel menager', 'Titre': 'reemployer reutiliser materiau inerte chantier chantier'}


### On charge le modèle 

In [67]:
# Charger le modèle
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

## Sauvegarde des embeddings

On va sauvegarder les embeddings en gardant le nom des champs.

In [123]:
def encoder_embeddings(data, model, output_file):

    # Fonction pour encoder chaque texte
    def encoder_texte(texte):
        return model.encode(texte)

    # Pour chaque entrée dans les données, encoder tous les champs texte
    for entry in data:
        # print("DEBUG : entry = ", entry)
        for champ, valeur in entry[1].items():
            # print("DEBUG : champ =", champ, ", valeur = ", valeur)
            if isinstance(valeur, str):  # S'assurer que la valeur est une chaîne de caractères
                # Calculer l'embedding
                embedding = encoder_texte(valeur)
                # Remplacer le texte par l'embedding
                entry[1][champ] = embedding.tolist()

    # Sauvegarder les embeddings dans un fichier
    with open(output_file, 'wb') as file:
        pickle.dump(data, file)

    return data

Commençons par calculer un embedding sur deux solutions :

In [124]:
path_embedding_file = "embeddings/model_final_magb.pkl"
solutions_embeddings = encoder_embeddings(df_solutions_clean, model, path_embedding_file)

In [119]:
# Pour voir notre embedding sur notre première solution.
for champ, valeur in solutions_embeddings[0][1].items():
    print(champ," : ", valeur)

Titre technologie  :  [-0.0695968046784401, -0.023653583601117134, -0.014692571014165878, -0.016898155212402344, 0.035704951733350754, -0.04877614974975586, 0.05672469362616539, 0.011476914398372173, -0.03712873160839081, -0.006696578115224838, 0.09654846042394638, 0.17023248970508575, -0.014968565665185452, 0.20785005390644073, 0.014589879661798477, -0.04955305531620979, 0.09957949072122574, 0.14145119488239288, -0.1957101672887802, 0.07550809532403946, -0.03613649681210518, -0.04644988104701042, 0.029459575191140175, -0.040124889463186264, -0.15210013091564178, -0.06397569924592972, 0.024910619482398033, 0.03387347236275673, 0.09107264131307602, 0.2496613711118698, 0.14505918323993683, 0.13652732968330383, 0.012056484818458557, 0.004843400791287422, 0.014716026373207569, -0.04563531279563904, -0.042045388370752335, -0.011440101079642773, -0.06426902115345001, 0.18276232481002808, 0.14868991076946259, -0.0740155354142189, -0.038239482790231705, 0.048926711082458496, -0.045451316982507

In [126]:
print(solutions_embeddings[0])

[2, {'Titre technologie': [-0.13877683877944946, -0.12198255956172943, -0.0159507617354393, 0.09502963721752167, -0.014843178912997246, 0.056268174201250076, 0.01649797149002552, -0.06368330121040344, 0.09359575808048248, 0.04101523384451866, 0.011926131322979927, 0.023840101435780525, -0.006083064712584019, -0.030989522114396095, 0.06731655448675156, -0.10868291556835175, 0.003249475732445717, 0.03398096561431885, -0.18035078048706055, 0.010271741077303886, -0.01647617667913437, 0.044262371957302094, 0.010300523601472378, 0.032754793763160706, -0.017926856875419617, -0.07374593615531921, -0.0046538966707885265, 0.11228595674037933, 0.10994794219732285, -0.01565842144191265, 0.02726741135120392, 0.049292560666799545, -0.0034133305307477713, 0.049437470734119415, 0.05939017981290817, -0.06210329383611679, -0.07927514612674713, 0.002498700050637126, -0.083034448325634, -1.1862721294164658e-05, 0.0950688049197197, -0.08507353067398071, -0.0183509960770607, 0.028124375268816948, -0.0685971

## Inférence

In [139]:
# Fonction appelé par notre utilisateur
def model_find_solution(description, secteur, model, embeddings_file_path, seuil_similarite, min_sol, weights) :
    # !!!!!!!!!!!! ICI ON A CHOISI DE NE PAS INCLURE LE SECTEUR DANS NOTRE MODELE, COMMENTER LA LIGNE CI-DESOSUS POUR CHANGER CELA --------------
    secteur = ""
    # -------------------------

    # On commence par concaténer notre secteur et notre description.
    requete = secteur + ". " + description

    # Ensuite on applique notre pré-processing
    clean_requete = clean_text(requete, remove_numbers=False)
    clean_requete_vecteur =  model.encode(clean_requete)

    # On va lire notre fichier d'embeddings 
    with open(embeddings_file_path, "rb") as fIn:
        solutions_embeddings = pickle.load(fIn)


    solutions_similarities = []
    # Maintenant pour chaque solution on va garder notre meilleur similarité cosinus avec notre requete. De plus nous ajouton un poids à chaque champs de notre solution,
    # Description * 1.3, Titre * 0.7, et 1 pour tous les autres.
    # On se retrouve donc avec une liste [[id_sol, max_similarité], ...]

    # Pour chaque entrée dans les données, encoder tous les champs texte
    for entry in solutions_embeddings:
        max_similarity = 0
        id_solution = entry[0]
        for champ, valeur_vecteur in entry[1].items():
            # Coeficient multiplicateur 
            weight = weights[champ]
            # On calcul la similarité
            similarity = util.pytorch_cos_sim(valeur_vecteur, clean_requete_vecteur)*weight
            # On met à jour le max de similarité
            if similarity > max_similarity :
                max_similarity = similarity
        solution_similarity = [id_solution, max_similarity]
        solutions_similarities.append(solution_similarity)
    #print("DEBUG : ",solutions_similarities)
    
    
    # On trie notre liste de solution par ordre croissant
    solutions_similarities.sort(key=lambda x: x[1], reverse=True)

    # On sélectionne les 5 meilleures solutions
    top_5_solutions = [sol[0] for sol in solutions_similarities[:5]]

    # On applique le seuil de similarité aux solutions restantes
    remaining_solutions = [sol[0] for sol in solutions_similarities[5:] if sol[1] > seuil_similarite]

    # On concatène les deux listes de solutions
    final_solutions = top_5_solutions + remaining_solutions

    # On retourne que les solutions et non la similarité
    return final_solutions

In [140]:
weights = {"Titre":1, "Description":1, "Application":1, "Bilan énergétique":1, "Titre technologie":1, "Description technologie":1}

In [142]:
model_find_solution("C'est quoi la HP flottante ?", "", model, path_embedding_file, 0.7, 5, weights)

[724, 835, 1533, 1608, 1595]

## Tester sur dataset Kerdos

In [ ]:
# Lire le fichier CSV
df_testset = pd.read_csv("data/dataset_test_Kerdos.csv")

def test_accuracy(embedding_path, dataset=df_testset, top_n=1) :
    accuracy = 0
    for i in range(1,len(dataset)):
        predictions = model_PAT("", dataset['Description'][i], embedding_path)
        print("---------------------------------------")
        print("Valeur attendu : ", dataset["id_solution"][i])
        print(predictions)
        if (dataset["id_solution"][i] in predictions[:top_n]):
            accuracy += 1/len(dataset)
        else :
            print("mal prédit : ", dataset["id_solution"][i])
    return accuracy